In [1]:
import numpy as np
from keras.layers import Input,Add,Conv2D,Dense,ZeroPadding2D,Activation,MaxPooling2D,Flatten
from keras.models import Model
from keras.utils import to_categorical
from keras.datasets import mnist, cifar10

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_st

In [2]:
import gzip
import numpy as np
CLIP_MIN = -0.5
CLIP_MAX = 0.5

In [3]:
cd sadl-master

[Errno 2] No such file or directory: 'sadl-master'
/home/ict520c/notebooks/sadl-master


In [5]:
import keras
from keras.datasets import mnist
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten
from keras.models import Sequential

path = 'original_data/'
files = ['train-images-idx3-ubyte.gz','train-labels-idx1-ubyte.gz'
,'t10k-images-idx3-ubyte.gz','t10k-labels-idx1-ubyte.gz']
paths = [path+each for each in files]
with gzip.open(paths[1],'rb') as imgpath:
    y_train = np.frombuffer(imgpath.read(),np.uint8,offset=8)
with gzip.open(paths[0],'rb') as imgpath:
    x_train = np.frombuffer(imgpath.read(),np.uint8,offset=16).reshape(len(y_train),28,28,1)
with gzip.open(paths[3],'rb') as imgpath:
    y_test= np.frombuffer(imgpath.read(),np.uint8,offset=8)
with gzip.open(paths[2],'rb') as imgpath:
    x_test = np.frombuffer(imgpath.read(),np.uint8,offset=16).reshape(len(y_test),28,28,1)
        


x_test = x_test.reshape(-1,28,28,1)
x_test = x_test.astype("float32")
x_test = (x_test / 255.0) - (1.0 - 0.5)

x_train = x_train.reshape(-1,28,28,1)
x_train= x_train.astype("float32")
x_train= (x_train / 255.0) - (1.0 - 0.5)

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

model = Sequential()

# 选取6个特征卷积核，大小为5∗5(不包含偏置),得到66个特征图，每个特征图的大小为32−5+1=2832−5+1=28，
# 也就是神经元的个数由10241024减小到了28∗28=78428∗28=784。
# 输入层与C1层之间的参数:6∗(5∗5+1)
model.add(Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1)))

# 这一层的输入为第一层的输出，是一个28*28*6的节点矩阵。
# 本层采用的过滤器大小为2*2，长和宽的步长均为2，所以本层的输出矩阵大小为14*14*6。
model.add(MaxPooling2D(pool_size=(2, 2)))

# 本层的输入矩阵大小为14*14*6，使用的过滤器大小为5*5，深度为16.本层不使用全0填充，步长为1。
# 本层的输出矩阵大小为10*10*16。本层有5*5*6*16+16=2416个参数
model.add(Conv2D(16, kernel_size=(5, 5), activation='relu'))

# 本层的输入矩阵大小10*10*16。本层采用的过滤器大小为2*2，长和宽的步长均为2，所以本层的输出矩阵大小为5*5*16。
model.add(MaxPooling2D(pool_size=(2, 2)))

# 本层的输入矩阵大小为5*5*16，在LeNet-5论文中将这一层称为卷积层，但是因为过滤器的大小就是5*5，#
# 所以和全连接层没有区别。如果将5*5*16矩阵中的节点拉成一个向量，那么这一层和全连接层就一样了。
# 本层的输出节点个数为120，总共有5*5*16*120+120=48120个参数。
model.add(Flatten())
model.add(Dense(120, activation='relu'))

# 本层的输入节点个数为120个，输出节点个数为84个，总共参数为120*84+84=10164个 (w + b)
model.add(Dense(84, activation='relu'))

# 本层的输入节点个数为84个，输出节点个数为10个，总共参数为84*10+10=850
model.add(Dense(10, activation='softmax'))

model.compile(loss=keras.metrics.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=128, epochs=20, verbose=1, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

W0912 16:06:20.303728 140465949632320 module_wrapper.py:137] From /home/ict520c/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0912 16:06:20.318116 140465949632320 module_wrapper.py:137] From /home/ict520c/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0912 16:06:20.387714 140465949632320 module_wrapper.py:137] From /home/ict520c/.local/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0912 16:06:20.462375 140465949632320 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 10s 171us/step - loss: 0.3258 - acc: 0.9050 - val_loss: 0.0960 - val_acc: 0.9703
Epoch 2/20
60000/60000 [==============================] - 7s 119us/step - loss: 0.0916 - acc: 0.9718 - val_loss: 0.0710 - val_acc: 0.9783
Epoch 3/20
60000/60000 [==============================] - 7s 117us/step - loss: 0.0651 - acc: 0.9804 - val_loss: 0.0544 - val_acc: 0.9823
Epoch 4/20
60000/60000 [==============================] - 9s 143us/step - loss: 0.0528 - acc: 0.9842 - val_loss: 0.0429 - val_acc: 0.9865
Epoch 5/20
60000/60000 [==============================] - 9s 147us/step - loss: 0.0432 - acc: 0.9864 - val_loss: 0.0485 - val_acc: 0.9834
Epoch 6/20
60000/60000 [==============================] - 8s 135us/step - loss: 0.0388 - acc: 0.9875 - val_loss: 0.0411 - val_acc: 0.9862
Epoch 7/20
60000/60000 [==============================] - 8s 133us/step - loss: 0.0316 - acc: 0.9904 - val_loss: 0.0455 -

In [7]:
   model.save("./model/lenet_model.h5")

model.summary()

In [11]:
print([layer.name for layer in model.get_layer('model').layers])



ValueError: No such layer: model

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 6)         156       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 6)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 16)          2416      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 4, 4, 16)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 120)               30840     
_________________________________________________________________
dense_3 (Dense)              (None, 84)                10164     
__________

In [ ]:
x = Image.open('orig.png')